In [1]:
#Definindo finais de semana
mes = "07"
lista_fds = ['06','07','13','14','20','21','27','28']

# Funções e Pacotes

In [2]:
# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#definindo quantidade de linhas e colunas visiveis
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Manipulação de Strings
import unidecode # => unidecode(string)
import re # Regex

# Repositorio - Diretorio
import os
import shutil

In [3]:
#######################-FUNÇÕES DE CONTAGEM
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)
        
def tabela_cont(linha, coluna):
    a = df.groupby([linha, coluna]).size().unstack()
    for i in a.columns:
        a[i].fillna(0, inplace = True)
        a[i] = a[i].apply(lambda x: int(x))
    return a
    print('='*100)

def tabela_cont_percent(linha, coluna, opcao):
    result = tabela_cont(linha, coluna)
    if opcao == 'Linha':
        return round(result.div(result.sum(axis=1), axis=0) * 100, 2)
    elif opcao == 'Coluna':
        return round(result.div(result.sum(), axis=1) * 100, 2)
    else:
        return 'Opção invalida'

In [4]:
# Funções de salvar arquivo
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

In [5]:
def read_file(file_path):
    # 1. Ler o arquivo txt
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    
    # 2. Remover todas as linhas que contêm "Mensagem apagada"
    lines = [line for line in lines if "Mensagem apagada" not in line]
    
    # 3. Corrigir as mensagens que estão em várias linhas para que fiquem em uma única linha
    corrected_lines = []
    current_message = ""
    
    for line in lines:
        # Se a linha começa com o padrão de data e hora, ela é uma nova mensagem
        if re.match(r"^\[\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}\]", line):
            if current_message:  # Se há uma mensagem anterior, adicione-a à lista
                corrected_lines.append(current_message)
            current_message = line.strip()  # Comece uma nova mensagem
        else:
            current_message += " " + line.strip()  # Adicione o conteúdo da linha atual à mensagem em progresso
    
    # Adiciona a última mensagem, se houver
    if current_message:
        corrected_lines.append(current_message)
    
    # Salvando arquivo txt
    with open('processed.txt', "w", encoding="utf-8") as file:
        for line in corrected_lines:
            file.write(line + "\n")
    
    # Lendo arquivo txt e transformando em dataframe cada mensagem
    with open('processed.txt', 'r', encoding='utf-8') as file:
        content = file.read().splitlines()
    
    return content

In [6]:
def sample_prob(nivel):
    '''
    data = df_prob[df_prob['controle'] == nivel]
    tamanho = len(data)
    try:
        if tamanho == 0:
            data_exibir = str('0 itens do prob' + nivel)
        if tamanho == 1:
            data_exibir = data['mensagem'].sample(1)
        if tamanho == 2:
            data_exibir = data['mensagem'].sample(2)
        if tamanho > 5:
            data_exibir = data['mensagem'].sample(5)
    except:
        data_exibir = 'erro'
    '''
    return 'nada'

# =========================================

# Carregando base

In [7]:
df = pd.DataFrame({'chat': read_file('_chat.txt') })
#df = pd.read_excel('df_prob.xlsx')[['chat']]; os.remove("df_prob.xlsx")

#df['chat'] = df['chat'].apply(lambda x: x.replace('/05/2024, ','/05/2024 '))

df['controle'] = np.nan
df = df[['controle','chat']]
df_prob = pd.DataFrame(columns = ['controle','chat'])

print(len(df))
display(df.head())

541


,controle,chat
0,NaN,[23/07/2024 10:10:03] Débora Philippi Bressane...
1,NaN,[23/07/2024 10:11:02] Flávio Maia Castilho: 23...
2,NaN,[23/07/2024 10:12:58] Eduardo Arbache Bezerra:...
3,NaN,[23/07/2024 10:40:04] Luciano Angelo Richetti:...
4,NaN,[23/07/2024 10:59:58] Manoel Célio Moura Junio...


# Dia de envio da mensagem

In [8]:
dia = []
for i in range(0, len(df['chat'])):
    dia.append(df['chat'][i][1:11]) #pegando exatamente aonde parte da data esta
df['dia'] = dia
#print(sorted(df['dia'].unique()))
print("Quantidade de mensagens por dia:")
count_freq(df, 'dia')

Quantidade de mensagens por dia:


,Freq,Perc
dia,,
24/07/2024,262,48.428835
23/07/2024,164,30.314233
25/07/2024,115,21.256932


# Nome Anestesista

In [9]:
df['nome'] = np.nan
count = 0
for i in range(0, len(df['chat'])):
    loc = df['chat'][i].find(']')+1 #digito data
    loc2 = df['chat'][i][loc+1:].find(':')+1 #digito nome
    nome = df['chat'][i][loc+1:loc+loc2]
    nome = nome.upper().strip().replace("  ", " ")
    nome = unidecode.unidecode(nome)
    df['nome'][i] = nome
    count += 1
    #print(count, "->", unidecode.unidecode(chat[i][loc+1:loc+loc2].upper()))

print("Profissionais que mandaram mensagem:")
df['nome'] = df['nome'].apply(lambda x: x.replace('  ',' '))
sorted(df['nome'].unique())

Profissionais que mandaram mensagem:


['ALESSANDRE LUIZ BRAGA JUNIOR',
 'ALEXANDRE AMARAL LUCAS',
 'ALEXANDRE BERNARDINO',
 'ALINE FRANCHINI GOULART LIMA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURAO BARRETO',
 'ANA JUNG DOCKO',
 'ANDRE DA COSTA AGUIAR VENTURA',
 'ANDRE LUIS XAVIER FRANCO',
 'ANDREIA LOUREIRO MORI',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ANTONIO ALVARO DIAS LIMA DA CUNHA NETO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BEATRIZ ANTIPOU DOS SANTOS',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUCE YE MAN CHOW',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAMILA CHIARANTANO HARRISON',
 'CAMILA RESENDE DE PAULA',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO CORREA GRILO',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'DANIEL MASUZAKI WONG',
 'DANIELLEN LINS LOURENCO',
 'DARLAN JUNIOR SCHNEIDER',
 'DEBORA ALVES AUDE',
 'DEBORA PHILIPPI BRESSANE',
 'DENIO LOPES DE ALMEIDA',
 'DIEGO DA CUNHA

# Mensagens escrevidas e enviadas

In [10]:
mensagem = []
for i in range(0, len(df)):
    digito_data = df['chat'][i].find(']') #aonde acaba a data
    digito_nome = df['chat'][i][digito_data+1:].find(':')
    mensagem_escrevida = df['chat'][i][digito_data+digito_nome+3:]
    mensagem.append(mensagem_escrevida)
    
df['mensagem'] = mensagem
df['mensagem'] = df['mensagem'].apply(lambda x: x.upper() )
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('-',' - '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('(',' ( '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace(')',' ) '))

print('Mensagens Enviadas:')
df['mensagem'].sample(3)

Mensagens Enviadas:


510      25/07 VILLA  -  J. HUSSEIN ENTRADA 7:00
147               23/07 ITAIM DÉBORA SAÍDA 17:50
498    24/07 ANALIA  -  ALESSANDRE ENTRADA 07:00
Name: mensagem, dtype: object

# =========================================

# Data do Plantão

In [11]:
df['data'] = np.nan

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'DATA_DO_PLANTÃO'

#salvando
df_prob = df[df['controle'] == 'DATA_DO_PLANTÃO']
print("data probs:", len(df_prob))
sample_prob('DATA_DO_PLANTÃO')

df = df[df['controle'] != 'DATA_DO_PLANTÃO']
print("datas ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

count_freq(df, 'data')
#print("Dias mensionados nas mensagens:")
#print(sorted(df['data'].unique()))

data probs: 0
datas ok: 541


,Freq,Perc
data,,
24/07,260,48.059150
23/07,165,30.499076
25/07,101,18.669131
22/07,4,0.739372
23/7,3,0.554529
24/7,3,0.554529
24/06,2,0.369686
24/08,1,0.184843
25/06,1,0.184843


# Unidades

In [12]:
df['unidade'] = ''
df['Q_unidades'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    
    #Unidades escrita com a bunda
    lista_avi = ["CENTRAL","AVICENA","AVICENNA","AVICCENNA"]
    for i_avi in lista_avi:
        mensagem = mensagem.replace(i_avi, 'AVICCENA')
        
    mensagem = mensagem.replace('Sta Isabel', 'SANTA')
    
    lista_unidades = re.findall(r'ALPHA|ANALIA|AVICCENA|ITAIM|JABA|MATERNIDADE|MORUMBI|OSASCO|VILLA|VILA|VNS|SANTA', mensagem)
    quantidade_unidades = len(lista_unidades)
    
    df['Q_unidades'][i] = quantidade_unidades
    
    if quantidade_unidades == 1: #capturando apenas com uma unidade
        df['unidade'][i] = lista_unidades[0].replace('  ',' ').upper().strip()
    elif quantidade_unidades > 1: #com mais de uma unidade
        df['unidade'][i] = lista_unidades
        df['controle'][i] = 'DESLOCADO'
    else: #sem nenhuma unidade
        df['controle'][i] = 'UNIDADES'
        
print("quantidades de unidades presentes nas mensagens:")
count_freq(df, 'Q_unidades')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'UNIDADES']])
print("unidades probs:", len(df_prob))
sample_prob('UNIDADES')

df = df[df['controle'] != 'UNIDADES']

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
#print("unidade ok:", len(df)) 
#print("unidade ok deslocado:", len(df[df['controle'] == 'DESLOCADO']))

quantidades de unidades presentes nas mensagens:


,Freq,Perc
Q_unidades,,
1,510,94.269871
2,27,4.990758
0,4,0.739372


unidades probs: 4


# Deslocados

In [13]:
#preparando para reconhecer corretamente
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCADA','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCAMENTO','DESLOCADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DESLOCADO', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes > 0:
        df['controle'][i] = 'DESLOCADO'
        
count_freq(df, 'controle')
# OBS: Plantões de deslocado não são mais excluido

,Freq,Perc
controle,,
DESLOCADO,30,100.0


# (Retirada) Distancia e Saude

In [14]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA
df['controle2'] = np.nan
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSADA','DISPENSADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSA ','DISPENSADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISTANCIA|D[0-9]', mensagem)
    quantidade_observacoes = len(observacoes)

    if quantidade_observacoes >= 1:
        df['controle'][i] = 'DISP_SAUDE_DIST'
        if quantidade_observacoes == 1:
            df['controle2'][i] = observacoes[0]
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISP_SAUDE_DIST']])
print("unidades probs:", len(df_prob))
sample_prob('DISP_SAUDE_DIST')

df = df[df['controle'] != 'DISP_SAUDE_DIST']
print("unidade ok:", len(df)) 

print(df_prob['mensagem'][df_prob['controle'] == 'DISP_SAUDE_DIST'].sample(2))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,29,69.047619
DISP_SAUDE_DIST,13,30.952381


unidades probs: 17
unidade ok: 524
312    24/07 CENTRAL  -  ÁLVARO DISPENSADO
93        23/07 SANTA  -  DIEGO DISPENSADO
Name: mensagem, dtype: object


In [15]:
for i in range(0, len(df)):
    mensagem = df['mensagem'][i].upper()
    regex_D = re.findall('D[0-9]|DISTANCIA', mensagem)
    
    if len(regex_D) == 0:
        continue
    else:
        df['controle'][i] == 'DISTANCIAS'

count_freq(df, 'controle')

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISTANCIAS']])
print("probs:", len(df_prob))
sample_prob('DISTANCIAS')

df = df[df['controle'] != 'DISTANCIAS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,29,100.0


probs: 17
ok: 524


# Extras

In [16]:
df["extra"] = np.nan

for i in range(0, len(df)):
    if 'EXTRA' in df['mensagem'][i].upper(): #se conter a palavra "extra"
        df['extra'][i] = 'extra' #marque na coluna extra como extra
        
print("Quantidade de registros Extra - Voluntario:")
count_freq(df, 'extra')      

Quantidade de registros Extra - Voluntario:


,Freq,Perc
extra,,


# Tipo entrada e saida

In [17]:
#Indentificando se é um registro de entrada ou saida
df['tipo'] = np.nan

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    entradas_saidas = re.findall(r'ENTRADA|SAIDA', mensagem)
    quantidade_entradas_saidas = len(entradas_saidas)
    
    if quantidade_entradas_saidas != 1: #capturando apenas com uma informação escrita
        df['controle'][i] = 'SEM_ENTRADA_OU_SAIDA'
    else: #restante com mais de uma informação
        df['tipo'][i] = entradas_saidas[0]
        
count_freq(df, 'tipo')
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'SEM_ENTRADA_OU_SAIDA']])
print("unidades probs:", len(df_prob))
sample_prob('SEM_ENTRADA_OU_SAIDA')

df = df[df['controle'] != 'SEM_ENTRADA_OU_SAIDA']
print("unidade ok:", len(df)) 

#print(df_prob['mensagem'][df_prob['controle'] == 'SEM_ENTRADA_OU_SAIDA'].sample(1))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

,Freq,Perc
tipo,,
ENTRADA,279,53.34608
SAIDA,244,46.65392


,Freq,Perc
controle,,
DESLOCADO,29,96.666667
SEM_ENTRADA_OU_SAIDA,1,3.333333


unidades probs: 18
unidade ok: 523
ok: 523


# -----------------------------------------

# Data

In [18]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = np.nan

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio

In [19]:
#arrumando data d/mm (6/10, não 06/10)
for i in range(0, len(df)):
    if df['data'][i][1] == '/':
        df['data'][i] = '0'+df['data'][i]
df['data'].unique()

array(['23/07', '23/7', '22/07', '24/07', '24/7', '24/06', '24/08',
       '25/07', '25/06', '04/07'], dtype=object)

In [20]:
#somando informações restantes na data: dd/MM/yyyy
for i in range(0, len(df)):
    df['data'][i] = df['data'][i][0:2] + "/"+ mes +"/2024" 
    
count_freq(df, 'data')

,Freq,Perc
data,,
24/07/2024,256,48.948375
23/07/2024,162,30.975143
25/07/2024,101,19.311663
22/07/2024,3,0.573614
04/07/2024,1,0.191205


In [21]:
df['sem_ou_fds'] = 'semana'

for i in lista_fds:
    datinha = i+"/"+mes+"/2024"
    df['sem_ou_fds'][df['data'] == datinha] = 'fds'

count_freq(df, 'sem_ou_fds')

,Freq,Perc
sem_ou_fds,,
semana,523,100.0


In [22]:
print(df.groupby(['data', 'sem_ou_fds']).size().unstack())

sem_ou_fds  semana
data              
04/07/2024       1
22/07/2024       3
23/07/2024     162
24/07/2024     256
25/07/2024     101


In [23]:
#df['data'].iloc[180] = '26/09/2022'

# Hora

In [24]:
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

In [25]:
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas == 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'HORA'
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORA']])
print("probs:", len(df_prob))
sample_prob('HORA')

df = df[df['controle'] != 'HORA']
print("ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,29,87.878788
HORA,4,12.121212


probs: 22
ok: 519


In [26]:
#adicionando 0 no primeiro algorismo dos valores h:mm
for i in range(0, len(df)):
    if df['hora'][i][1] == ":":
        df['hora'][i] = "0"+df['hora'][i]
        
df['hora'].unique()

array(['09:00', '07:00', '11:00', '06:00', '06:10', '11:20', '11:40',
       '13:00', '13:10', '13:15', '13:30', '12:20', '13:45', '13:55',
       '14:20', '14:25', '14:30', '14:40', '15:00', '15:10', '15:20',
       '15:35', '15:37', '11:15', '16:00', '16:15', '16:25', '16:35',
       '05:30', '16:50', '17:00', '17:10', '17:30', '17:35', '17:39',
       '17:40', '17:45', '17:50', '15:40', '18:00', '12:00', '18:08',
       '18:10', '18:15', '18:25', '18:37', '18:50', '19:00', '19:05',
       '19:10', '19:18', '19:35', '19:30', '19:45', '17:20', '19:50',
       '20:10', '20:00', '19:20', '20:25', '20:30', '20:40', '20:45',
       '20:35', '20:58', '21:00', '21:15', '21:20', '21:30', '21:55',
       '22:38', '22:40', '22:30', '22:50', '22:15', '21:45', '23:40',
       '23:45', '04:40', '04:45', '04:50', '05:00', '05:10', '05:15',
       '05:18', '05:20', '05:25', '05:45', '23:15', '06:20', '05:50',
       '06:30', '05:40', '07:20', '06:15', '09:36', '10:00', '10:30',
       '11:10', '12:

# Hora final

In [27]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [28]:
df['hora'].unique()

array(['09:00', '07:00', '11:00', '06:00', '06:10', '11:20', '11:40',
       '13:00', '13:10', '13:15', '13:30', '12:20', '13:45', '13:55',
       '14:20', '14:25', '14:30', '14:40', '15:00', '15:10', '15:20',
       '15:35', '15:37', '11:15', '16:00', '16:15', '16:25', '16:35',
       '05:30', '16:50', '17:00', '17:10', '17:30', '17:35', '17:39',
       '17:40', '17:45', '17:50', '15:40', '18:00', '12:00', '18:08',
       '18:10', '18:15', '18:25', '18:37', '18:50', '19:00', '19:05',
       '19:10', '19:18', '19:35', '19:30', '19:45', '17:20', '19:50',
       '20:10', '20:00', '19:20', '20:25', '20:30', '20:40', '20:45',
       '20:35', '20:58', '21:00', '21:15', '21:20', '21:30', '21:55',
       '22:38', '22:40', '22:30', '22:50', '22:15', '21:45', '23:40',
       '23:45', '04:40', '04:45', '04:50', '05:00', '05:10', '05:15',
       '05:18', '05:20', '05:25', '05:45', '23:15', '06:20', '05:50',
       '06:30', '05:40', '07:20', '06:15', '09:36', '10:00', '10:30',
       '11:10', '12:

In [29]:
#criando uma coluna datetime para fazer calculos de horas e minutos
df['hora2'] = df['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

#verificando se esta tudo certo
print(df['hora2'].dt.year.unique())
print(df['hora2'].dt.month.unique())
print(df['hora2'].dt.day.unique())
print(df['hora2'].dt.hour.unique())

for i in range(0, len(df)):
    minuto = int(df['hora'][i][3:])
    
    if minuto != 0 and minuto != 30:
        if minuto < 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (0) subtraindo quantos minutos tem entre o momento e HH:00
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-minuto))[11:-3]
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (30) acrecentando quantos minutos faltam para HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(30-minuto)))[11:-3]
            else:
                df['controle'][i] = 'Hora_final'
                
        elif minuto > 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (30) subtraindo quando minutos tem entre o momento e HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-(minuto-30)))[11:-3]   
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (60) acrecentando quantos minutos faltam para HH:60
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(60-minuto)))[11:-3]            
            else:
                df['controle'][i] = 'Hora_final'
        else:
            df['controle'][i] = 'Hora_final'
            
    elif minuto == 0 or minuto == 30:
        df['hora'][i] = str(df['hora2'][i])[11:-3] 
        
    else:
        df['controle'][i] = 'Hora_final'

df['hora'].unique()

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'Hora_final']])
print("probs:", len(df_prob))
sample_prob('Hora_final')

df = df[df['controle'] != 'Hora_final']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

[1900]
[1]
[1]
[ 9  7 11  6 13 12 14 15 16  5 17 18 19 20 21 22 23  4 10  1]
probs: 22
ok: 519


# Setor

In [30]:
#reconhendo em qual periodo do dia aconteceu o plantão
df['ENTRADA'] = ''
df['SAIDA'] = ''

df['PERIODO'] = ''

for i in range(0, len(df)):
    minuto = int(df['hora'][i][:2])
    
    if df['tipo'][i] == 'ENTRADA':
        df['ENTRADA'][i] = df['hora'][i]
        if minuto < 7: #periodo 6h
            df['PERIODO'][i] = '6h'
            df['SAIDA'][i] = '07:00'
        elif minuto >= 7 and minuto < 13: #periodo manha
            df['PERIODO'][i] = 'manha'
            df['SAIDA'][i] = '13:00'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['PERIODO'][i] = 'tarde'
            df['SAIDA'][i] = '19:00'
        else:
            df['controle'][i] = 'HORARIOS'
    elif df['tipo'][i] == 'SAIDA':
        df['SAIDA'][i] = df['hora'][i]
        if minuto >= 7 and minuto < 13: #periodo manha
            df['ENTRADA'][i] = '07:00'
            df['PERIODO'][i] = 'manha'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            if df['hora'][i] == '13:00':
                df['SAIDA'][i] = '13:00'
                df['ENTRADA'][i] = '07:00'
                df['PERIODO'][i] = 'manha'
            else:
                df['ENTRADA'][i] = '13:00'
                df['PERIODO'][i] = 'tarde'
        elif minuto >= 19: #periodo noturno
            df['ENTRADA'][i] = '19:00'
            df['PERIODO'][i] = 'noturno saida'
        else:
            df['controle'][i] = 'HORARIOS'
    else:
        df['controle'][i] = 'HORARIOS'

df['controle'][df['ENTRADA'] == df['SAIDA']] = 'HORARIOS'

count_freq(df, 'PERIODO')
count_freq(df, 'controle')

df['periodo'] = np.nan
df['periodo'][df['PERIODO'] == '6h'] = 1
df['periodo'][df['PERIODO'] == 'manha'] = 2
df['periodo'][df['PERIODO'] == 'tarde'] = 3
df['periodo'][df['PERIODO'] == 'noturno saida'] = 4

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORARIOS']])
print("probs:", len(df_prob))
sample_prob('HORARIOS')

df = df[df['controle'] != 'HORARIOS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
PERIODO,,
manha,196,37.764933
tarde,111,21.387283
noturno saida,104,20.038536
6h,103,19.845857
,5,0.963391


,Freq,Perc
controle,,
DESLOCADO,29,52.727273
HORARIOS,26,47.272727


probs: 48
ok: 493


# Duração do Plantão

In [31]:
df['DURAÇÃO'] = ''

for i in range(0, len(df)):
    diferenca_tempo = datetime.strptime(df['SAIDA'][i], '%H:%M') - datetime.strptime(df['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    df['DURAÇÃO'][i] = horas + ':' + minutos
    
df['DURAÇÃO'][df['DURAÇÃO'] == '00:00'] = '06:00'    

df['DURAÇÃO'].unique()

array(['04:00', '06:00', '02:00', '01:00', '04:30', '05:00', '00:30',
       '05:30', '01:30', '02:30', '03:00', '03:30'], dtype=object)

# Checagem Unidades:

In [32]:
# alterando JABA e AVVICENA
def replace_values(value):
    if value == 'JABA':
        return 'JABAQUARA'
    elif value == 'AVICCENA':
        return 'CENTRAL DO TATUAPE'
    elif value == 'SANTA':
        return 'SANTA ISABEL'
    else:
        return value
    
def replace_values2(value):
    if isinstance(value, list):
        return [replace_values(v) for v in value]
    else:
        return replace_values(value)

# Aplicar a função à coluna 'setor'
df['unidade'] = df['unidade'].apply(lambda x: replace_values2(x))

In [33]:
# Cria uma nova Série sem as listas
serie_sem_listas = pd.Series([x for x in df['unidade'] if not isinstance(x, list)])

# Imprime os valores únicos da nova Série
print(serie_sem_listas.unique())

['JABAQUARA' 'ITAIM' 'SANTA ISABEL' 'VNS' 'ANALIA' 'VILLA' 'MORUMBI'
 'OSASCO' 'MATERNIDADE' 'ALPHA' 'CENTRAL DO TATUAPE']


In [34]:
# Cria uma lista com apenas os valores que são listas
listas = [x for x in df['unidade'] if isinstance(x, list)]

# Converte as listas em tuplas e adiciona elas a um conjunto para obter apenas as listas distintas
listas_distintas = set(tuple(lista) for lista in listas)

# Imprime as listas distintas
for lista in listas_distintas:
    print(list(lista))


['ANALIA', 'JABAQUARA']
['VNS', 'ANALIA']
['VILLA', 'JABAQUARA']
['VILLA', 'MORUMBI']
['ITAIM', 'MATERNIDADE']
['ITAIM', 'VNS']
['VNS', 'MATERNIDADE']
['MORUMBI', 'ITAIM']
['MATERNIDADE', 'VNS']
['ITAIM', 'MORUMBI']
['OSASCO', 'VILLA']
['VILLA', 'VILLA']
['OSASCO', 'JABAQUARA']
['MORUMBI', 'JABAQUARA']
['OSASCO', 'ANALIA']
['JABAQUARA', 'VILLA']
['JABAQUARA', 'ITAIM']


# Setor (2)

In [35]:
df['setor'] = np.nan

for i in range(0, len(df)):
    if df['sem_ou_fds'][i] == 'semana':
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'SEMANA - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'SEMANA - NOTURNO - SAIDA'
        else:
            df['setor'][i] = 'VERIFICAR'

    elif df['sem_ou_fds'][i] == 'fds': 
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'FDS - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'FDS - C.CIRURGICO ROTINA'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'FDS - SAIDA'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'FDS - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'  

    else:
        df['setor'][i] = 'VERIFICAR'
        
df['setor'] = df['setor'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
count_freq(df, 'setor')

,Freq,Perc
setor,,
SEMANA - MANHA - C.CIRURGICO,190,38.539554
SEMANA - TARDE - C.CIRURGICO,111,22.515213
SEMANA - 6HS,103,20.892495
SEMANA - NOTURNO - SAIDA,89,18.052738


# *****************************************

# Pareamento

In [36]:
print(len(df))
print(len(df_prob))

493
48


In [37]:
df_prob_pareado = pd.DataFrame(columns=df.columns)
df_prob_pareado

,controle,chat,dia,nome,mensagem,data,unidade,Q_unidades,controle2,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,periodo,DURAÇÃO,setor


In [38]:
count_ok = 0
count_erro = 0

for nome in sorted(df['nome'].unique()):
    df_filter = df[df['nome'] == nome]
    for data in sorted(df_filter['data'].unique()):
        df_filter = df[(df['nome'] == nome) & (df['data'] == data)]

        if len(df_filter) >= 2:
            qtd_entrada = len(df_filter[df_filter['tipo'] == 'ENTRADA'])
            qtd_saida = len(df_filter[df_filter['tipo'] == 'SAIDA'])

            if qtd_entrada == qtd_saida:
                print(nome, '|', data)
                count_ok += 1
        else:
            # Procurar se o registro foi mandado com o erro:
            df_prob_filter = df_prob[(df_prob['nome'] == nome) & (df_prob['data'] == data)]

            # ERRO PARCIAL
            if len(df_prob_filter) > 0: 
                print(nome, '|', data, '|', 'PARCIAL')
                df_prob['controle'][df_prob_filter.index] = 'erro_parcial'
                
            # ERRO DE PERDA TOTAL
            else: 
                print(nome, '|', data, '|', 'ERRO')
                df_filter['controle'] = 'erro_total'
                df_prob_pareado = pd.concat([df_prob_pareado, df_filter])

            count_erro += 1
    print('='*50)

ALEXANDRE AMARAL LUCAS | 24/07/2024
ALEXANDRE BERNARDINO | 23/07/2024 | PARCIAL
ALEXANDRE BERNARDINO | 24/07/2024
ALINE FRANCHINI GOULART LIMA | 23/07/2024 | ERRO
AMANDA KIYOKO SOUZA KOSSAKA | 24/07/2024
ANA CLARA MOURAO BARRETO | 23/07/2024 | ERRO
ANA CLARA MOURAO BARRETO | 24/07/2024 | ERRO
ANA CLARA MOURAO BARRETO | 25/07/2024 | ERRO
ANA JUNG DOCKO | 24/07/2024
ANDRE DA COSTA AGUIAR VENTURA | 24/07/2024
ANDRE DA COSTA AGUIAR VENTURA | 25/07/2024 | ERRO
ANDRE LUIS XAVIER FRANCO | 25/07/2024 | ERRO
ANDREIA LOUREIRO MORI | 24/07/2024
ANDREIA LOUREIRO MORI | 25/07/2024 | ERRO
ANNA BEATRIZ ARANHA TREVISAN | 23/07/2024
ANNA BEATRIZ ARANHA TREVISAN | 24/07/2024
ANTONIO ALVARO DIAS LIMA DA CUNHA NETO | 23/07/2024
ARTHUR OLIVEIRA FACCHINI | 23/07/2024
ARTHUR OLIVEIRA FACCHINI | 24/07/2024
ARTHUR RODRIGUES TORRELIO | 23/07/2024 | ERRO
ARTHUR RODRIGUES TORRELIO | 24/07/2024 | ERRO
ARTHUR RODRIGUES TORRELIO | 25/07/2024 | ERRO
AUGUSTO DE CASTRO SANTOS QUEIROZ | 23/07/2024 | ERRO
AUGUSTO DE CAST

# (0) Salvando problemas de entrada e saida

In [39]:
print(count_ok)
print(count_erro)

158
171


In [40]:
salvando_nome_arquivo('df_entrada_saida.xlsx', df_prob_pareado)

156


# Ultimos ajustes:

In [41]:
df = df[['chat', 'dia', 'nome', 'mensagem', 'controle', 'data', 'unidade',
       'extra', 'tipo', 'ENTRADA', 'SAIDA', 'DURAÇÃO', 'PERIODO','periodo',
         'setor', 'sem_ou_fds']]

# =========================================

# Deslocados:

In [42]:
df['controle'][(df['PERIODO'] == '6h') & (df['controle'] == 'DESLOCADO')] = 'DESLOCADO 6H'
count_freq(df, 'controle')

,Freq,Perc
controle,,
DESLOCADO,20,68.965517
DESLOCADO 6H,9,31.034483


In [43]:
df['unidade2'] = df['unidade']

for i in range(0, len(df)):
    if isinstance(df.loc[i, 'unidade'], list):
        df['unidade'][i] = df['unidade'][i][0]

df['unidade'][df['unidade'] == 'VILA'] = 'VILLA'
df['unidade'][df['unidade'] == 'JABA'] = 'JABAQUARA'
df['unidade'][df['unidade'] == 'SANTA'] = 'SANTA ISABEL'

sorted(df['unidade'].unique())
count_freq(df, 'unidade')

,Freq,Perc
unidade,,
ITAIM,87,17.647059
ANALIA,84,17.038540
JABAQUARA,67,13.590264
OSASCO,65,13.184584
MORUMBI,62,12.576065
VILLA,53,10.750507
SANTA ISABEL,22,4.462475
VNS,19,3.853955
ALPHA,16,3.245436


# (1) Salvando tudo: gabs & giov

In [44]:
salvando_nome_arquivo('df_gabs.xlsx', df)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

493
.
48


# =========================================

# Dispensados:

In [45]:
df = pd.read_excel('df_giov.xlsx')[['chat','dia','nome','mensagem']]
df = df.drop_duplicates(subset=['chat'])
df = df.sort_values(['nome', 'dia'])
print(len(df))
df.sample(3)

48


,chat,dia,nome,mensagem
21,[25/07/2024 07:15:34] Fábio Antônio de Souza O...,25/07/2024,FABIO ANTONIO DE SOUZA OLIVEIRA,25/07 ITAIM FABIO ENTRADA 7HS
38,[24/07/2024 18:49:21] Patrick Alexander Sauer ...,24/07/2024,PATRICK ALEXANDER SAUER SCHUES,24/07 ANÁLIA - PATRICK SAÍDA 18:45
32,[24/07/2024 07:29:20] Felipe de Oliveira Nunes...,24/07/2024,FELIPE DE OLIVEIRA NUNES,24/07 SANTA ISABEL - NUNES SAÍDA 7:00 ( PLA...


In [46]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA

df['controle'] = np.nan

#preparando para reconhecer corretamente

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISPENSADA', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes == 0:
        df['controle'][i] = 'PROB'
        
df['controle'].value_counts() 

#salvando
df_prob = df[df['controle'] == 'PROB']
df_prob.reset_index(drop=True, inplace=True)
print("unidades probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("unidade ok:", len(df)) 

unidades probs: 40
unidade ok: 8


In [47]:
#DATA DO PLANTÃO
df['controle'] = ''
df['data'] = ''

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
        df['controle'][i] = 'NORM'
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'PROB'

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'NORM']])
df_prob.reset_index(drop=True, inplace=True)
print("data probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("datas ok:", len(df)) 

df['data'] = df['data']+"/2024"
sorted(df['data'].unique())

data probs: 40
datas ok: 8


['22/07/2024', '23/07/2024', '24/07/2024', '25/07/2024']

In [48]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = ''

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio
    
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

#Retirando aqueles que contem horas no registro da mensagem
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas >= 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
        df['controle'][i] = 'PROB'
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'OK'
        
print(df['controle'].value_counts())

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'OK']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] == 'OK']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

controle
OK      7
PROB    1
Name: count, dtype: int64
probs: 41
ok: 7


In [49]:
print(len(df_prob))
print(len(df))

41
7


# (2) Salvando tudo: giov & dispensados

In [50]:
df = df.drop_duplicates(subset=['chat'])

os.remove("df_giov.xlsx")
salvando_nome_arquivo('df_giov.xlsx', df_prob)
print('.')
salvando_nome_arquivo('df_dispensados.xlsx', df)

41
.
7


# ===========================================

# Ultima Contagem

In [51]:
print(len(pd.DataFrame({'chat': read_file('_chat.txt') })))
print('.')
print(len(pd.read_excel('df_dispensados.xlsx')))
print('.')
print(len(pd.read_excel('df_gabs.xlsx')))
print('.')
print(len(pd.read_excel('df_giov.xlsx')))

541
.
7
.
493
.
41
